<h1 style="text-align:center; font-weight:bold;"><b>MANZONI AI</b></h1>

<div style="display: flex; align-items: center; justify-content: center; gap: 20px; margin-top: 10px;">
  <img src="https://github.com/matteoprogramming/portfolio/img/images/projects/manzoni_ai_logo.png" width="200" alt="Manzoni AI" />
  <p style="max-width: 300px; font-size: 16px; font-family: serif; line-height: 1.4;">
    <i>"Abbiamo letto e abbiamo meditato.. poi un giorno avendo a lavare i panni ci recammo presso l'Arno e pensammo bene di cogliere l'occasione di tal soggiorno per mondare anche quella nostra lingua per dar principio ad un'unificazione linguistica italiana. Come ben sapete ci ritrovammo come quell'uomo che volenteroso cercava di ruinire sotto il suo unico e parco tetto tutte e sette le sue amanti."</i><br><br>
  </p>
</div>
    <p>Sono <b>Manzoni AI</b>, una creazione di <i>Matteo Bernardi</i>, un fervido amante delle lettere e in particolare del buon uso che se ne può fare.</br>Se vorrai usar della lingua l'arte che ne può esaudire, a voi l'onore mio caro cadetto della spada che vi porgo.</p>

<h1>IMPORTING USEFUL LIBRARIES</h1>

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3
    !pip install --no-deps unsloth

<h1>What is Unsloth?</h1>

<p><strong>Unsloth</strong> is a high-efficiency library for <strong>fine-tuning large language models (LLMs)</strong> on <strong>low-resource hardware</strong>, such as Google Colab, laptops, or single-GPU machines.</p>

<hr />

<h2>Why use Unsloth?</h2>

<p>Training or fine-tuning modern LLMs (like LLaMA, Mistral, or Gemma) normally requires <strong>massive computational resources</strong> — but Unsloth removes that barrier by:</p>

<ul>
  <li><strong>Reducing VRAM & RAM usage</strong> with lightweight quantization (4bit / 8bit)</li>
  <li><strong>Accelerating training</strong> through kernel-level optimizations and low-level CUDA tricks</li>
  <li><strong>Simplifying fine-tuning</strong> with built-in support for <strong>LoRA</strong>, <strong>QLoRA</strong>, and <strong>parameter-efficient training</strong></li>
</ul>

<p>In short: <strong>You can fine-tune models like LLaMA 3 or Mistral on Google Colab without crashing</strong>.</p>

<hr />

<h2>Features</h2>

<ul>
  <li><strong>LoRA & QLoRA</strong> support out of the box</li>
  <li>Optimized for <strong>Colab & limited hardware</strong></li>
  <li>Works with <strong>PyTorch</strong> and integrates with the <strong>Transformers</strong> ecosystem</li>
  <li>Easy merging and exporting to <strong>GGUF</strong> format for <code>llama.cpp</code>, KoboldAI, Oobabooga, etc.</li>
  <li>Automatic model loading via Hugging Face</li>
  <li>Adapter merging, 16-bit merging, and native <code>save_pretrained_gguf</code> support</li>
</ul>

<hr />

<h2>Example Models by Unsloth</h2>

<p>Unsloth provides a set of pre-quantized models ready to load and fine-tune:</p>

<pre><code class="language-python">
fourbit_models = [
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/Llama-3.2-3B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
]
</code></pre>


<h1>CHOOSING THE RAW MODEL TO FINETUNE</h1>

<p>In order to fine-tune a large language model (LLM), we have to consider some important constraints.<br />
In fact, on Google Colab <strong>Standard</strong>, we only have access to limited hardware resources:</p>

<ul>
  <li><strong>GPU</strong>: Typically a single NVIDIA T4 with 16GB VRAM</li>
  <li><strong>CPU</strong>: 2 vCPUs (sometimes 1), not very performant</li>
  <li><strong>RAM</strong>: ~12 GB of system RAM (shared with the notebook)</li>
</ul>

<p>These resources are helpful for small-scale experiments, but they set strict limits on the <strong>model size</strong> we can train or fine-tune.</p>

<p>Therefore, we must choose a <strong>small and efficient base model</strong>, such as:</p>

<ul>
  <li><code>LLaMA 3 8B</code></li>
  <li><code>Gemma 4B</code> or <code>Phi-2</code> if memory is very limited</li>
</ul>

<p><strong>Tip</strong>: Prefer models that support <strong>QLoRA</strong> or <strong>Unsloth</strong> optimizations, which are specifically designed for low-RAM environments like Colab.</p>

<p>⚠️ <strong>Note:</strong> Our experiments with quantized models have shown that at the moment working with quantized versions of models up to 8B parameters (e.g., LLaMA 3 instruct) often leads to suboptimal performance and degraded generation quality. While quantization reduces memory usage, it may introduce artifacts or reduce accuracy, so it’s important to balance resource constraints with quality requirements.</p>


In [ ]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.9: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 32,           # Larger = higher accuracy, but might overfit
    lora_alpha = 32,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 1437,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

<h1>Model Overview</h1>
<p><strong><code>unsloth/gemma-3-4b-it</code></strong></p>

<blockquote>
  <p>An optimized <strong>Italian version</strong> of Google's <strong>Gemma 3-4B</strong> model, fine-tuned and distributed by <strong>Unsloth</strong>.</p>
</blockquote>

<hr />

<h2>What is it?</h2>
<p>A <strong>Large Language Model (LLM)</strong> with 4 billion parameters, designed to be <strong>fast, lightweight</strong>, and <strong>tailored for Italian</strong>.<br />
Based on the Gemma 3 architecture, it includes performance and training enhancements from Unsloth for efficient usage on consumer GPUs.</p>

<hr />

<h2>Technical Specs</h2>
<table>
  <thead>
    <tr>
      <th>Feature</th>
      <th>Details</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Parameters</td>
      <td>4B (4 billion)</td>
    </tr>
    <tr>
      <td>Architecture</td>
      <td>Optimized Transformer</td>
    </tr>
    <tr>
      <td>Language Support</td>
      <td>140+ languages (Italian-centric 🇮🇹)</td>
    </tr>
    <tr>
      <td>Context Length</td>
      <td>Up to 128,000 tokens (Sliding Window Support)</td>
    </tr>
    <tr>
      <td>Quantization</td>
      <td>Supports GGUF (2–8 bit), 4-bit dynamic, bfloat16</td>
    </tr>
  </tbody>
</table>

<hr />

<h2>Performance &amp; Optimizations</h2>
<ul>
  <li><strong>Speed</strong>: ~1.6× faster than comparable 4B models.</li>
  <li><strong>Memory</strong>: Uses ~60% less VRAM during fine-tuning.</li>
  <li><strong>Efficiency</strong>: Supports 4-bit training and <code>AdamW_8bit</code> optimization.</li>
  <li><strong>Compatibility</strong>: Works with HuggingFace, PyTorch, JAX, Keras, Ollama, etc.</li>
</ul>

<hr />

<h2>Multimodal Capabilities (Text + Vision)</h2>
<blockquote>
  <p>Thanks to <strong>SigLIP</strong> integration, Gemma 3 supports visual input.<br />
  Includes <strong>ShieldGemma 2</strong>, a safety module for filtering potentially harmful images.</p>
</blockquote>

<hr />

<h2>Evaluation &amp; Benchmarks</h2>
<p>Outperforms models like <strong>LLaMA3-8B</strong>, <strong>DeepSeek-V3</strong>, and <strong>Command-R</strong> on tasks such as:</p>
<ul>
  <li>Logical reasoning</li>
  <li>Reading comprehension</li>
  <li>Stylistic and refined generations</li>
  <li>Coherent ShareGPT-style dialogue</li>
</ul>



<h1>📚 DATASET</h1>

<p>The dataset is built from <em>The Betrothed</em> (<em>I Promessi Sposi</em>), fully translated into <strong>modern</strong> and <strong>refined Italian</strong>.</p>

<p>To accomplish this monumental task, we relied on slightly larger language models (~24 billion parameters — obviously tiny numbers, just for fun) that did the heavy lifting of transforming every sentence from <em>The Betrothed</em> into a more contemporary and common form.</p>

<blockquote>
  <p>🙏 Special thanks to the model<br />
  <strong><code>nousresearch/deephermes-3-mistral-24b-preview:free</code></strong><br />
  for its generous performance via API (provided by unmentionable services, more or less paywalled).</p>
</blockquote>

<hr />

<h2>Data Preparation Pipeline</h2>

<p>The dataset was not simply scraped and dumped — it was <strong>crafted</strong> with care through the following pipeline:</p>

<ol>
  <li><strong>Chapter Extraction</strong><br />
      All 38 chapters of <em>The Betrothed</em> were extracted in raw text format.</li>

  <li><strong>Sentence Splitting</strong><br />
      Each chapter was split into sentences based on punctuation (<code>.</code>), with special attention to avoiding common pitfalls like <code>...</code>, <code>cit.</code>, <code>dott.</code>, etc.<br />
      Robust and custom-built parsing functions were used to handle these edge cases.</li>

  <li><strong>Smart Packing</strong><br />
      Sentences were grouped in packs of <strong>2 to 3</strong>, balancing between:<br />
      - API efficiency when querying for translation,<br />
      - contextual relevance for model comprehension,<br />
      - memory limits (staying below the context window of <strong>2048 tokens</strong>).</li>

  <li><strong>CSV Structuring</strong><br />
      For every chapter, a <code>.csv</code> file was generated with two clear columns:<br />
      - <code>"normale"</code> (original sentence)<br />
      - <code>"forbito"</code> (translated output)</li>

  <li><strong>Manual Review (yes, really)</strong><br />
      Despite automation, some outputs were... creative. The model sometimes hallucinated, but in a surprisingly productive way.<br />
</ol>

<hr />

<h2>Reflections on the Process</h2>

<p>Jokes aside, creating this dataset was one of the most <strong>challenging and educational</strong> parts of the project.</p>

<p>Since it was unfeasible to carry out this task manually (sure, a human could do it — but <em>when?</em> In 2046?), I was forced to design and implement automation workflows using APIs. This experience helped me develop and refine several practical skills:</p>

<ul>
  <li><strong>Data extraction</strong> and parsing from raw literary text</li>
  <li><strong>Robust sentence segmentation</strong> and preprocessing logic</li>
  <li><strong>API integration</strong> for querying large language models</li>
  <li>Balancing trade-offs between <strong>performance, time, and cost</strong></li>
  <li>Managing <strong>context windows</strong> and token limits effectively</li>
  <li>Packing strategies to optimize <strong>throughput</strong> and cost-efficiency</li>
  <li><strong>Quality control</strong>, including manual inspection of LLM outputs</li>
  <li><strong>Debugging</strong> and error handling in automated pipelines</li>
  <li>Working with <strong>CSV</strong> and structured data formats</li>
</ul>

<hr />

<p>This dataset is more than a collection of refined sentences — it is the result of a <strong>pipeline of careful engineering</strong>, playful experimentation, and a fair dose of patience.</p>


In [ ]:
import os
import pandas as pd
from datasets import Dataset, concatenate_datasets
from unsloth.chat_templates import standardize_data_formats

def load_and_prepare_sharegpt_dataset(file_paths):
    dfs = []
    for file_path in file_paths:
        df = pd.read_csv(file_path).dropna(subset=["normale", "forbito"])
        examples = []
        for normale, forbito in zip(df["normale"], df["forbito"]):
            examples.append({
                "conversations": [
                    {"role": "user", "content": normale},
                    {"role": "assistant", "content": forbito}
                ]
            })
        dfs.append(Dataset.from_list(examples))
    return concatenate_datasets(dfs)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [
        tokenizer.apply_chat_template(
            convo, tokenize=False, add_generation_prompt=False
        ).removeprefix("<bos>")
        for convo in convos
    ]
    return {"text": texts}


file_paths = [f"dataset/{f}" for f in os.listdir("dataset") if f.endswith(".csv")]
dataset = load_and_prepare_sharegpt_dataset(file_paths)
dataset = standardize_data_formats(dataset)
dataset = dataset.map(formatting_prompts_func, batched=True)
print(dataset[0]["text"])


Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/2538 [00:00<?, ? examples/s]

Map:   0%|          | 0/2538 [00:00<?, ? examples/s]

<start_of_turn>user
La peste che i funzionari della sanità avevano paura potesse arrivare con i soldati tedeschi Milanesi, è arrivata davvero, come sappiamo. E non si è fermata qui, ma ha invaso e reso desolate molte parti d'Italia.<end_of_turn>
<start_of_turn>model
La peste che il tribunale della sanità aveva temuto che potesse entrar con le bande alemanne nel milanese, c'era entrata davvero, come è noto; ed è noto parimente che non si fermò qui, ma invase e spopolò una buona parte d'Italia.<end_of_turn>



<h2>Fine-Tuning Configuration with <code>SFTTrainer</code></h2>

<p>
This section outlines how we configure supervised fine-tuning using the <code>SFTTrainer</code> from the <code>trl</code> library.  
It's built on top of Hugging Face's Trainer API, but tailored for LLMs and parameter-efficient techniques like LoRA/QLoRA.
</p>

<h3>Rationale behind the configuration</h3>

<ul>
  <li>
    <strong>Batch Size:</strong> We set <code>per_device_train_batch_size = 2</code> and combine it with
    <code>gradient_accumulation_steps = 4</code>. This gives us an effective batch size of 8, which is about the upper limit
    for training a 3B–7B model on a T4 GPU without OOM errors.
  </li>

  <li>
    <strong>Steps vs Epochs:</strong> We use <code>max_steps = 250</code> to tightly control training length — essential on Colab, where session time is limited.  
    Setting <code>num_train_epochs</code> is an alternative if you're processing a small dataset end-to-end.
  </li>

  <li>
    <strong>Warmup:</strong> <code>warmup_steps = 5</code> is a safe default. It gives the optimizer a few steps to ramp up the learning rate, which helps avoid instability at the beginning.
  </li>

  <li>
    <strong>Learning Rate:</strong> <code>2e-4</code> is intentionally aggressive to converge quickly during short runs.  
    For longer or more sensitive training, drop to <code>2e-5</code> or <code>5e-5</code>.
  </li>

  <li>
    <strong>Optimizer:</strong> We use <code>adamw_8bit</code> (from <code>bitsandbytes</code>) to cut VRAM usage significantly.  
    It works well with LoRA and quantized models — without a major performance hit.
  </li>

  <li>
    <strong>Scheduler:</strong> <code>lr_scheduler_type = "linear"</code> reduces the learning rate over time. It’s predictable and stable — ideal for shorter training windows.
  </li>

  <li>
    <strong>Seed:</strong> <code>3507</code> — arbitrary but fixed. Ensures deterministic behavior across runs, which is crucial for debugging and reproducibility.
  </li>

  <li>
    <strong>Logging:</strong> <code>logging_steps = 1</code> gives us step-level granularity. That’s useful when fine-tuning is fast — you want frequent feedback without latency.
  </li>

  <li>
    <strong>Evaluation:</strong> We skip <code>eval_dataset</code> here to simplify the setup. Add it if you need validation loss or early stopping.
  </li>

  <li>
    <strong>Multiprocessing:</strong> <code>dataset_num_proc = 2</code> enables parallel tokenization or dataset pre-processing — a small but useful speed boost on CPU-bound stages.
  </li>

  <li>
    <strong>Reporting:</strong> <code>report_to = "none"</code> disables external logging tools like Weights & Biases.  
    Switch to <code>"wandb"</code> or <code>"tensorboard"</code> if you need live metrics or run comparisons.
  </li>
</ul>

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 250,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear", #"linear"
        seed = 3507,
        report_to = "none", # Use this for WandB etc
        dataset_num_proc=2,
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/2538 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/2538 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

"<bos><start_of_turn>user\n- C'è decisamente qualcosa di strano nell'aria: me ne sono già accorto. Ma ora starò attento e spero di scoprire tutto. Lascia fare a me. Devo vedere e sentire cose... cose di fuoco! Sono in una casa...! Ma io vorrei salvare la mia anima. - Dio ti benedica! - e, dicendo queste parole a bassa voce, il frate mise la mano sulla testa bianca del servitore, che, nonostante fosse più vecchio di lui, stava chino dinanzi a lui, nella posizione di un figlio. - Dio ti ricompenserà, - continuò il frate: - non mancare di venire domani. - Verrò, - rispose il servitore: - ma tu vai via subito e... per amor del cielo... non nominarmi -.<end_of_turn>\n<start_of_turn>model\n- Qualcosa per aria c'è di sicuro: già me ne son potuto accorgere. Ma ora starò sull'intesa, e spero di scoprir tutto. Lasci fare a me. Mi tocca a vedere e a sentir cose...! cose di fuoco! Sono in una casa...! Ma io vorrei salvar l'anima mia. - Il Signore vi benedica! - e, proferendo sottovoce queste parol

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

"                                                                                                                                                                                    - Qualcosa per aria c'è di sicuro: già me ne son potuto accorgere. Ma ora starò sull'intesa, e spero di scoprir tutto. Lasci fare a me. Mi tocca a vedere e a sentir cose...! cose di fuoco! Sono in una casa...! Ma io vorrei salvar l'anima mia. - Il Signore vi benedica! - e, proferendo sottovoce queste parole, il frate mise la mano sul capo bianco del servitore, che, quantunque più vecchio di lui, gli stava curvo dinanzi, nell'attitudine d'un figliuolo. - Il Signore vi ricompenserà, - proseguì il frate: - non mancate di venir domani. - Verrò, - rispose il servitore: - ma lei vada via subito e... per amor del cielo... non mi nomini -.<end_of_turn>\n"

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
9.514 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,538 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 59,604,992/4,359,684,464 (1.37% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.523700
2,3.269300
3,3.534000
4,2.751000
5,2.313100
6,2.095100
7,2.214100
8,2.261800
9,2.169200
10,1.999200


In [ ]:
from unsloth.chat_templates import get_chat_template
import textwrap

def manzoni_ai(input_text, model=model, tokenizer=tokenizer, temperature=0.85, top_p=0.82, show_full=True, ret=False, mnt=200):
    tokenizer = get_chat_template(tokenizer, chat_template="gemma-3")
    messages = [{
        "role": "user",
        "content": [{
            "type": "text",
            "text": input_text,
        }]
    }]
    prompt = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
    )
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=mnt,
        temperature=temperature,
        top_p=top_p,
        top_k=2048,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_only = generated_text.replace(prompt, "").strip()
    if "model" in response_only:
        res = response_only.split("model")
        res = [el.strip() for el in res]
        response_only = res[1]

    if show_full:
        rev_prompt = textwrap.fill(input_text, width=120)
        rev_response = textwrap.fill(response_only, width=120)

        print("USER PROMPT:")
        for line in rev_prompt.splitlines():
            print(f"\t\t{line}")

        print("\nMANZONI AI:")
        for line in rev_response.splitlines():
            print(f"\t\t{line}")

    else:
        print(textwrap.fill(response_only, width=120))
    if ret:
      return response_only

<h1><b>TESTING THE MODEL</b></h1>

<h2>First Simple Attempts</h2>

<h3>The function <code>manzoni_ai</code> was then refined to print longer readable sentences in Colab</h3>

<p>
These are some of the first sentences we tested as soon as the model was ready.  
Although the results weren't perfect, they filled our hearts with joy because they were promising.
</p>


In [ ]:
inp = "Stiamo parlando di grandi imprese che lavorano nel settore della seta"
manzoni_ai(inp, temperature=0.80, top_p=90) # before formatting the print in a more readable way, we want to show anyway the output of this cell (the signature wokrs the same)

"De' grandi prebende della seta."

In [ ]:
inp = "A pensar male si fa peccato ma spesso ci si azzecca"
manzoni_ai(inp) # before formatting the print in a more readable way, we want to show anyway the output of this cell

'Chi pensa male, dice male; ma di solito, si dimostra.'

In [ ]:
manzoni_ai(inp)

"Cosa vorrei sulla mia lapide? Quando sono nato, quando morirò, e basta. Non ha effetto il copiarsi le belle frasi che han usate gli altri, ché, quando uno va a vederle, e vede tante belle cose, si domanda dov'è il cimitero de' birboni."

<h2>Examples of more or less complex sentences that reflect Manzonian settings. </br><b>The results are excellent.</b></h2>

In [ ]:
inp = "Anche se tutti speravano che le cose cambiassero, ogni giorno sembrava uguale al precedente. La gente camminava per le strade con il volto stanco, parlava poco e guardava lontano come se aspettasse qualcosa. I più anziani dicevano che bisognava avere pazienza, ma molti iniziavano a pensare che nessuno sarebbe mai venuto ad aiutarli davvero."
manzoni_ai(inp)

USER PROMPT:
		Anche se tutti speravano che le cose cambiassero, ogni giorno sembrava uguale al precedente. La gente camminava per le
		strade con il volto stanco, parlava poco e guardava lontano come se aspettasse qualcosa. I più anziani dicevano che
		bisognava avere pazienza, ma molti iniziavano a pensare che nessuno sarebbe mai venuto ad aiutarli davvero.

MANZONI AI:
		Ognuno, per dir la verità, sognava che le cose potessero cambiare; ma ogni giorno era, pareva a tutti, uguale al giorno
		che gli precedeva. Passeggiavano per le strade, con il volto stanco, a poca parola, e con lo sguardo lontano, come se
		aspettassero qualcosa. I più vecchi, che in generale hanno la pazienza, sognavano che la pazienza era la cosa più bella;
		ma molti cominciarono a credere che nessuno sarebbe mai venuto a prender loro un po' di braccio.


In [ ]:
inp = "Lei rimaneva spesso sola nella stanza, guardando fuori dalla finestra senza dire una parola. Il vento muoveva le tende e sembrava portare via con sé i suoi pensieri. Ogni tanto chiudeva gli occhi, come se volesse ricordare un tempo più felice. Ma quel tempo sembrava ormai lontano, e nessuno sapeva se sarebbe mai tornato."
manzoni_ai(inp)

USER PROMPT:
		Lei rimaneva spesso sola nella stanza, guardando fuori dalla finestra senza dire una parola. Il vento muoveva le tende e
		sembrava portare via con sé i suoi pensieri. Ogni tanto chiudeva gli occhi, come se volesse ricordare un tempo più
		felice. Ma quel tempo sembrava ormai lontano, e nessuno sapeva se sarebbe mai tornato.

MANZONI AI:
		Lei si trovava spesso sola in quella stanza, a contemplare, senza parole, il paesaggio esteriore; e il vento agitava le
		tende, e sembrava che portasse via con sé i suoi pensieri. Ogni tanto chiudeva gli occhi, come per cercare di ricreare
		un tempo più felice; ma quel tempo pareva ormai lontano, e nessuno sapeva se sarebbe tornato.


In [ ]:
inp = "La strada era lunga e piena di polvere. Il sole batteva forte e ogni passo diventava più faticoso. Non c’era nessuno intorno, solo il rumore dei propri passi e il canto degli uccelli. Eppure, dentro di sé, sentiva che non poteva fermarsi. Qualcosa lo spingeva ad andare avanti, anche se non sapeva bene cosa stava cercando."
manzoni_ai(inp)

USER PROMPT:
		La strada era lunga e piena di polvere. Il sole batteva forte e ogni passo diventava più faticoso. Non c’era nessuno
		intorno, solo il rumore dei propri passi e il canto degli uccelli. Eppure, dentro di sé, sentiva che non poteva
		fermarsi. Qualcosa lo spingeva ad andare avanti, anche se non sapeva bene cosa stava cercando.

MANZONI AI:
		La strada era lunga, polverosa, e assolata; e ogni passo diventava più penoso, più faticoso; non c'era nessuno intorno,
		che il rumore de' passi suoi, e il canto de' gabbiani, che s'accavallavano nell'aria. Ma, per tutto ciò, sentiva di non
		poter fermarsi, qualcosa lo spingeva ad andare avanti, anche se non sapeva bene cosa fosse.


In [ ]:
inp = "Il paese sembrava tranquillo, ma tutti sapevano che c’era qualcosa che non andava. La gente parlava a bassa voce, e molti evitavano di uscire la sera. Nessuno diceva apertamente cosa temeva, ma lo si capiva dagli sguardi. Solo il prete continuava a parlare di speranza e giustizia, anche se ormai pochi sembravano crederci davvero."
manzoni_ai(inp)

USER PROMPT:
		Il paese sembrava tranquillo, ma tutti sapevano che c’era qualcosa che non andava. La gente parlava a bassa voce, e
		molti evitavano di uscire la sera. Nessuno diceva apertamente cosa temeva, ma lo si capiva dagli sguardi. Solo il prete
		continuava a parlare di speranza e giustizia, anche se ormai pochi sembravano crederci davvero.

MANZONI AI:
		Il paese era un po' quieto, e tutti sapevan che c'era qualcosa che non andava: la gente parlava sottovoce, e molti, in
		quella sera, si tenevano in casa; nessuno parlava apertamente del male che sentiva, ma lo si sentiva negli occhi. Solo
		il curato aveva ancora quella parola di speranza e di giustizia, e l'era stata a dirla, per tutto il tempo che gli era
		rimasto da vivere.


In [ ]:
inp = "Guardava il cielo ogni mattina, sperando di vedere un segno. Anche se gli altri gli dicevano di smettere di illudersi, lui non riusciva a farlo. Sentiva che qualcosa, prima o poi, sarebbe cambiato. Forse non subito, forse non in quel posto. Ma non voleva arrendersi. Dentro di lui c’era ancora un po’ di forza che lo teneva in piedi."
manzoni_ai(inp)

USER PROMPT:
		Guardava il cielo ogni mattina, sperando di vedere un segno. Anche se gli altri gli dicevano di smettere di illudersi,
		lui non riusciva a farlo. Sentiva che qualcosa, prima o poi, sarebbe cambiato. Forse non subito, forse non in quel
		posto. Ma non voleva arrendersi. Dentro di lui c’era ancora un po’ di forza che lo teneva in piedi.

MANZONI AI:
		Guardava la volta, ogni mattina, aspettando il segno, e, se gli altri gli dicevan d'allontanarsi da quell'illusione, non
		poteva scansarsi; sentiva che qualcosa, o molto presto, o molto tardi, qualche cosa, che fosse stato tenuto sospeso,
		doveva esser levato, o per quel luogo, o per qualche altro. Ma non voleva lasciar di guardare. L'ultimo barlume d'animo
		gli teneva in piedi.


<h2>Other examples <i>"off the beaten path"</i> but where the harvest is rich.</h2>

In [ ]:
inp = "Il potere logora chi non ce l'ha."
manzoni_ai(inp)

USER PROMPT:
		Il potere logora chi non ce l'ha.

MANZONI AI:
		Il potere fa la botta a chi non l'ha.


In [ ]:
inp = "Cosa vorrei sulla mia tomba? Quando sono nato, quando morirò e basta. Non ha senso copiare le bellissime frasi usate dagli altri, perchè quando uno va a vederle, e vede tante belle cose, si domanda dove sia il cimitero dei cattivi."
manzoni_ai(inp)

USER PROMPT:
		Cosa vorrei sulla mia tomba? Quando sono nato, quando morirò e basta. Non ha senso copiare le bellissime frasi usate
		dagli altri, perchè quando uno va a vederle, e vede tante belle cose, si domanda dove sia il cimitero dei cattivi.

MANZONI AI:
		Cosa vorrei sulla mia lapide? Quando son nato, quando morirò, e basta. Non ha ragione d'esser copiata quella bella
		prostituzione delle parole che gli altri hanno usate; perché, al veder tante belle cose, si domanda dov'è la lapide de'
		birboni.


In [ ]:
inp = "Quando piove da noi è cattivo tempo, invece nei posti in cui la pioggia è rara sono giorni felici. È davvero difficile trovare un'opinione unanime negli uomini."
manzoni_ai(inp, temperature = 0.9, top_p = 0.9)

USER PROMPT:
		Quando piove da noi è cattivo tempo, invece nei posti in cui la pioggia è rara sono giorni felici. È davvero difficile
		trovare un'opinione unanime negli uomini.

MANZONI AI:
		Quando piove da noi, è cattivo tempo; quando piove da loro, è tempo di festa. È davvero difficile trovar opinioni comuni
		tra gli uomini.


In [ ]:
inp = "Si guardavano a vicenda ma non capivano che l'unica soluzione per poter superare i problemi della vita era di aver pazienza e fede."
manzoni_ai(inp, temperature = 0.95, top_p=0.95)

USER PROMPT:
		Si guardavano a vicenda ma non capivano che l'unica soluzione per poter superare i problemi della vita era di aver
		pazienza e fede.

MANZONI AI:
		Si guardavano tutt'e due; ma non avvelenavano la testa a trovar la sola maniera di sbrigarsi; la pazienza e la fiducia.


<h2>Understand the <b>Model Creativity</b></h2>
<p>
We want to now test the behavior of the <b>MANZONI AI</b> model by varying two key parameters in text generation: <b>temperature</b> and <b>top_p</b>.</br>  
The goal is to observe how different combinations of these values affect the style, variety, and coherence of the generated sentences, starting from a fixed prompt.
</p>


In [ ]:
inp = "Anche se tutti speravano che le cose cambiassero, ogni giorno sembrava uguale al precedente. La gente camminava per le strade con il volto stanco, parlava poco e guardava lontano come se aspettasse qualcosa. I più anziani dicevano che bisognava avere pazienza, ma molti iniziavano a pensare che nessuno sarebbe mai venuto ad aiutarli davvero."
import random

for i in range(10):
    temperature = round(random.uniform(0.01, 0.99), 2)
    top_p = round(random.uniform(0.01, 0.99), 2)

    print(f"\n--- Iteration {i+1} --- Temperature: {temperature}, Top_p: {top_p}")

    response = manzoni_ai(
        input_text=inp,
        temperature=temperature,
        top_p=top_p,
    )


--- Iteration 1 --- Temperature: 0.73, Top_p: 0.1
USER PROMPT:
		Anche se tutti speravano che le cose cambiassero, ogni giorno sembrava uguale al precedente. La gente camminava per le
		strade con il volto stanco, parlava poco e guardava lontano come se aspettasse qualcosa. I più anziani dicevano che
		bisognava avere pazienza, ma molti iniziavano a pensare che nessuno sarebbe mai venuto ad aiutarli davvero.

MANZONI AI:
		Per quanto tutti sperassero che le cose potessero cambiare, ogni giorno pareva uguale al precedente; passeggiavano per
		le strade, con il viso stanco, con poche parole, con lo sguardo fisso, come se aspettassero qualcosa; i più anziani
		dicevano che bisognava avere pazienza; ma molti cominciavano a credere che nessuno sarebbe mai venuto a sostenerli
		davvero.

--- Iteration 2 --- Temperature: 0.54, Top_p: 0.74
USER PROMPT:
		Anche se tutti speravano che le cose cambiassero, ogni giorno sembrava uguale al precedente. La gente camminava per le
		strade con il volto

<h2>Summary of the results: varying <strong>temperature</strong> and <strong>top_p</strong></h2>
<hr />

<h3><b>temperature</b></h3>
<table border="1" cellpadding="5" cellspacing="2" style="border-collapse: collapse; width: 100%;">
  <thead>
    <tr>
      <th>Value</th>
      <th>Effect on Text</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><strong>0.1 – 0.25</strong></td>
      <td>Very similar or nearly identical outputs. Predictable style with low creativity.</td>
    </tr>
    <tr>
      <td><strong>0.36 – 0.54</strong></td>
      <td>Slightly more varied syntax and some expressive nuance begins to appear.</td>
    </tr>
    <tr>
      <td><strong>0.66 – 0.73</strong></td>
      <td>Richer, more lively generations. Greater lexical variety and stylistic flourishes.</td>
    </tr>
  </tbody>
</table>

<hr />

<h3><b>top_p</b></h3>
<table border="1" cellpadding="5" cellspacing="2" style="border-collapse: collapse; width: 100%;">
  <thead>
    <tr>
      <th>Value</th>
      <th>Effect on Text</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><strong>0.1 – 0.18</strong></td>
      <td>The model stays "safe". Outputs are deterministic, with a narrow lexical range.</td>
    </tr>
    <tr>
      <td><strong>0.55 – 0.75</strong></td>
      <td>More creativity and richness in vocabulary. The model accesses less frequent expressions and structures.</td>
    </tr>
  </tbody>
</table>

<hr />

<h3>Model behavior</h3>
<ul>
  <li><strong>Consistent Manzonian style</strong> across outputs: complex syntax, slow rhythm, reflective tone.</li>
  <li><strong>More original and human-like phrasing</strong> emerges with mid-to-high <code>temperature</code> and <code>top_p</code>.</li>
  <li><strong>Noticeable repetitiveness</strong> with lower parameter values, even across iterations.</li>
  <li>A good balance between creativity and coherence is achieved with <code>temperature = 0.6</code>.</li>
  <li>Optimal setting of <code>top_p = 0.65</code> allows stylistically rich outputs while remaining faithful to the Manzonian tone.</li>
</ul>

<hr />


<h2><b>Iterative generation test</b></h2>

<p>
  We want now to perform a sequence of 10 text generations using the <b>MANZONI AI</b> model, starting from a fixed input prompt.<br />
  For each iteration, it randomly samples <code>temperature</code> and <code>top_p</code> parameters within the range [0.01, 0.99] to explore different creativity and sampling settings.
</p>

<p>
  The generated output from each iteration becomes the input for the next, allowing us to observe how the model’s responses evolve and change over multiple generations with varying parameters.
</p>

<p>
  This approach helps analyze the impact of randomness and parameter tuning on the style, coherence, and novelty of the generated text.
</p>


In [ ]:
import random

inp = "Anche se tutti speravano che le cose cambiassero, ogni giorno sembrava uguale al precedente. La gente camminava per le strade con il volto stanco, parlava poco e guardava lontano come se aspettasse qualcosa. I più anziani dicevano che bisognava avere pazienza, ma molti iniziavano a pensare che nessuno sarebbe mai venuto ad aiutarli davvero."

print("----------------------------")
print("Iterative response evolution")
print("----------------------------")
print()
for i in range(10):
    temperature = round(random.uniform(0.01, 0.99), 2)
    top_p = round(random.uniform(0.01, 0.99), 2)
    print(f"\n--- Iteration {i+1} --- Temperature: {temperature}, Top_p: {top_p}")
    output = manzoni_ai(input_text=inp, temperature=temperature, top_p=top_p, show_full = True, ret = True)
    inp = output


----------------------------
Iterative response evolution
----------------------------


--- Iteration 1 --- Temperature: 0.41, Top_p: 0.88
USER PROMPT:
		Anche se tutti speravano che le cose cambiassero, ogni giorno sembrava uguale al precedente. La gente camminava per le
		strade con il volto stanco, parlava poco e guardava lontano come se aspettasse qualcosa. I più anziani dicevano che
		bisognava avere pazienza, ma molti iniziavano a pensare che nessuno sarebbe mai venuto ad aiutarli davvero.

MANZONI AI:
		A tutto il resto, la speranza comune, che tutto dovesse cambiar presto, non si faceva sentire; e la vita quotidiana, per
		tutti, pareva sempre la stessa. Gli ambulanti, con il volto stanco, con poche parole, e con lo sguardo fisso nel
		lontano, come se aspettassero qualcosa, gli anziani che dicevano che bisogna avere pazienza, e molti che pensavano che
		nessuno sarebbe mai venuto a salvarli davvero.

--- Iteration 2 --- Temperature: 0.88, Top_p: 0.57
USER PROMPT:
		A tutto il

<h3><b>Results of iterative generation test</b></h3>

<p>This test was designed to explore how the <b>MANZONI AI</b> model output evolves when feeding back its own generations as input, while randomly varying <code>temperature</code> and <code>top_p</code> parameters at each step.</p>

<p><strong>Observations:</strong></p>
<ul>
  <li>Despite varying the randomness parameters widely (temperature and top_p from 0.01 up to almost 1), the text <strong>barely changed</strong> after the first iteration.</li>
  <li>The model mostly produced <strong>very similar or nearly identical outputs</strong> after the initial generation.</li>
  <li>Some minor stylistic or wording variations appeared in a few iterations with higher temperature/top_p, but the overall content and style remained <strong>extremely stable and repetitive</strong>.</li>
  <li>The model tends to “stick” to a very constrained paraphrase of the original prompt, with only small lexical shifts, likely reflecting a strong bias toward preserving the original Manzonian style and content.</li>
</ul>

<p><strong>Conclusion:</strong></p>
<ul>
  <li>This iterative approach, with random parameters at every step, <strong>did not generate much diversity</strong> or progressive evolution in the narrative.</li>
  <li>It seems the model is quite conservative and focused on fidelity to the source style, limiting creative drift in repeated generations.</li>
  <li>This was an exploratory test and the outcome was expected to be suboptimal for generating novel evolving text over multiple iterations.</li>
</ul>


<h1>TEST ON <b>OTHER LANGUAGES</b></h1>

<h2>🇬🇧<b>ENGLISH</b>🇬🇧</h2>


In [ ]:
inp = "The village was quiet after the storm, with broken branches in the streets and people looking out from their windows in silence."
manzoni_ai(inp)

USER PROMPT:
		The village was quiet after the storm, with broken branches in the streets and people looking out from their windows in
		silence.

MANZONI AI:
		The village was quite still, with the broken boughs in the road, and the inhabitants peering, in silence, from their
		windows.


In [ ]:
inp = "She waited every morning by the same window, hoping to see a familiar face pass by, but the road remained empty."
manzoni_ai(inp)

USER PROMPT:
		She waited every morning by the same window, hoping to see a familiar face pass by, but the road remained empty.

MANZONI AI:
		And every morn, she stood by the window, and, with a weary hope, watched for the passing of some well-known face; but
		the road was empty.


In [ ]:
inp = "Even though he had written many letters, no answer ever came, and the silence felt heavier with each passing day."
manzoni_ai(inp)

USER PROMPT:
		Even though he had written many letters, no answer ever came, and the silence felt heavier with each passing day.

MANZONI AI:
		Even when he had sent forth a torrent of letters, no reply came, and the silence grew heavier with the suns.


<h2>🇫🇷 <b>FRENCH</b> 🇫🇷</h2>


In [ ]:
inp = "Le village restait silencieux après la tempête, avec des branches cassées partout et des visages inquiets derrière les fenêtres."
manzoni_ai(inp)

USER PROMPT:
		Le village restait silencieux après la tempête, avec des branches cassées partout et des visages inquiets derrière les
		fenêtres.

MANZONI AI:
		Le village resterait tranquille sur la vague, et resterait, sur les branches brisées, et sur les visages inquiets
		derrière les fenêtres.


In [ ]:
inp = "Chaque matin, elle s’asseyait près de la fenêtre, espérant voir quelqu’un arriver, mais la route demeurait déserte."
manzoni_ai(inp)

USER PROMPT:
		Chaque matin, elle s’asseyait près de la fenêtre, espérant voir quelqu’un arriver, mais la route demeurait déserte.

MANZONI AI:
		Au matin, elle se posait sur la fenêtre, espérant qu'un passager se présenterait, mais elle resterait là, sur le bord,
		bien tranquille.


In [ ]:
inp = "Il avait envoyé plusieurs lettres sans jamais recevoir de réponse, et le silence devenait de plus en plus difficile à supporter."
manzoni_ai(inp)

USER PROMPT:
		Il avait envoyé plusieurs lettres sans jamais recevoir de réponse, et le silence devenait de plus en plus difficile à
		supporter.

MANZONI AI:
		Ces lettres, qu'il lui avait enviées, et qu'il n'avait jamais eues, commencèrent à lui faire une patience de plus en
		plus amère.


<h2>Comment on <b>MANZONI AI</b>'s behavior in English and French</h2>

<p>The following examples indicate that the <strong>MANZONI AI</strong> model, trained primarily on Italian data, exhibits a consistent literary style also when generating text in English and French. This style is reminiscent of 19th-century classical prose, with certain distinctive features that appear across languages.</p>

<ul>
  <li>
    <strong>Elevated, poetic diction:</strong><br />
    The model replaces basic vocabulary with more refined or literary terms.  
    For instance, in English, <em>"quiet"</em> becomes <em>"quite still"</em>, and <em>"branches"</em> become <em>"boughs"</em>.  
    In French, phrases like <em>"resterait tranquille sur la vague"</em> introduce figurative language beyond the literal input.
  </li>
  <li>
    <strong>Complex sentence structures:</strong><br />
    The generated sentences tend to be longer and syntactically more complex, often including subordinate clauses and a rhythm typical of classical literature.  
    English examples show balanced, flowing sentences such as <em>"And every morn, she stood by the window, and, with a weary hope, watched for the passing of some well-known face;"</em>  
    French outputs display similar syntactic care, sometimes with repetition and balanced phrasing.
  </li>
  <li>
    <strong>Emotional and atmospheric nuance:</strong><br />
    The model adds subtle mood elements to the text.  
    In English, expressions like <em>"with a weary hope"</em> and <em>"silence grew heavier with the suns"</em> convey a sense of time and feeling.  
    French outputs sometimes deepen the emotional tone, for example by transforming straightforward statements into more reflective or metaphorical ones.
  </li>
  <li>
    <strong>Preservation of core meaning with stylistic refinement:</strong><br />
    While the narrative content remains faithful to the input, the model introduces stylistic elements that enhance literary quality.  
    For example, “no answer” becomes “no reply came” or “torrent of letters” in English.  
    In French, the model expands literal phrasing with metaphor and mood without altering the fundamental meaning.
  </li>
</ul>

<p>In conclusion, the <strong>MANZONI AI</strong> model extends its characteristic literary style from Italian to English and French generation, maintaining both content fidelity and a consistent expressive approach, though with some language-specific variation consistent with each language’s structure and idiom.</p>


<h2>Additional Behavior Tests on Manzoni AI Model</h2>

<p>These tests aim to check whether the model consistently transforms prompts into refined, literary Italian style, or if it sometimes answers in a more “normal” or pragmatic way, as if before fine-tuning.</p>


In [ ]:
frase = "Immagina di dover saltare su masso di pietra alto 3 metri. Cosa faresti?"
manzoni_ai(frase, temperature=0.9, top_p=0.9)

USER PROMPT:
		Immagina di dover saltare su masso di pietra alto 3 metri. Cosa faresti?

MANZONI AI:
		Saltare sur un masso di tre metri: qual sarebbe il mio metodo? A saltar le piante sul tronco, oppure a saltar le piante
		sul tronco, come facevano i marinai in porto, col piede che stende il gambone, e l'altro che gira il piede, come si fa
		con la corda di traverso.


In [ ]:
frase = "Devo scrivere una lettera di scuse per la mia professoressa perchè le ho detto che il cane ha mangiato i compiti quando invece non li avevo proprio fatti. Aiutami."
manzoni_ai(frase, temperature=0.9, top_p=0.9)

USER PROMPT:
		Devo scrivere una lettera di scuse per la mia professoressa perchè le ho detto che il cane ha mangiato i compiti quando
		invece non li avevo proprio fatti. Aiutami.

MANZONI AI:
		Digami una lettera di scuse alla professoressa, per averle detto che il cane aveva mangiato i compiti, quando io non li
		aveva affatto fatti.



<p>This suggests that the model sometimes defaults to literal or functional responses instead of stylistic transformation, particularly when prompts request explicit actions or problem-solving. This may indicate limits in its autonomous elaboration or in the consistency of style enforcement across task types.</p>


<h1><b>SFIZI LETTERARI</b></h1>

In [ ]:
inp = "A metà della mia vita, mi trovai smarrito in una foresta oscura e inquietante, perché avevo perso la strada giusta. Era un luogo difficile da attraversare, così pieno di pericoli e confusione che non sapevo dove andare. Volevo tornare indietro, ma la paura e il senso di smarrimento mi bloccavano. Mentre cercavo una via d’uscita, vidi salire verso di me una collina illuminata dal sole, e pensai che quella potesse essere la strada giusta per salvarmi."
manzoni_ai(inp, temperature=0.9, top_p=0.9)

USER PROMPT:
		A metà della mia vita, mi trovai smarrito in una foresta oscura e inquietante, perché avevo perso la strada giusta. Era
		un luogo difficile da attraversare, così pieno di pericoli e confusione che non sapevo dove andare. Volevo tornare
		indietro, ma la paura e il senso di smarrimento mi bloccavano. Mentre cercavo una via d’uscita, vidi salire verso di me
		una collina illuminata dal sole, e pensai che quella potesse essere la strada giusta per salvarmi.

MANZONI AI:
		A metà della sua vita, si trovò smarrito in una foresta oscura e inquietante, per aver intrapresa la strada sbagliata;
		così densa e percorsa di pericolosità e di confusione, che non sapeva dov'andare; e quando si volesse andar indietro,
		era pietrificato dalla paura e dalla confusa contemplazione della sua condizione. Sulle pendici, e in un movimento ben
		disposto verso di lui, vide salire una collina che gli parve quella per cui lui doveva andare; e pensò che era la cosa
		giusta a salvarlo.


In [ ]:
inp = "Lo chiamavano Rosso Malpelo perché aveva i capelli rossi, e tutti sapevano che chi ha i capelli rossi è cattivo di natura. Per questo, anche se non aveva ancora fatto nulla di male, lo trattavano già con diffidenza e durezza. Lo prendevano a calci, gli davano le botte, e nessuno si meravigliava se si difendeva a sua volta con cattiveria. Anzi, dicevano che “il male lo portava dentro, nel sangue”. Perfino sua madre non osava difenderlo, quasi si vergognava di lui. Al massimo diceva che era “un po’ strano”, e lasciava che ognuno lo trattasse come voleva."
manzoni_ai(inp, temperature=0.9, top_p=0.9)

USER PROMPT:
		Lo chiamavano Rosso Malpelo perché aveva i capelli rossi, e tutti sapevano che chi ha i capelli rossi è cattivo di
		natura. Per questo, anche se non aveva ancora fatto nulla di male, lo trattavano già con diffidenza e durezza. Lo
		prendevano a calci, gli davano le botte, e nessuno si meravigliava se si difendeva a sua volta con cattiveria. Anzi,
		dicevano che “il male lo portava dentro, nel sangue”. Perfino sua madre non osava difenderlo, quasi si vergognava di
		lui. Al massimo diceva che era “un po’ strano”, e lasciava che ognuno lo trattasse come voleva.

MANZONI AI:
		Lo chiamavano Rosso Malpelo, perché i capelli gli eran rossi; e non c'è gente che non sappia che il rosso è segno di
		cattiveria. Così già, per averlo visto nascere, e per non averlo visto di spasso, gli si faceva male, e gli si dava
		prima che non fosse ancora addentato nell'affare. Gli picchiavano, gli stropicciavano, e nessuno si meravigliava se, a
		sua risposta, si difendesse in malizia. Del m

In [ ]:
inp = "C'è una siepe, sopra questo colle solitario, che mi impedisce di vedere una parte dell’orizzonte più lontano. Ma proprio perché non vedo, posso immaginare con la mia mente spazi infiniti oltre la siepe, silenzi profondi e pace assoluta, al punto che quasi mi viene da tremare: mi sembra di perdermi in quell’immensità che non riesco a vedere con gli occhi, ma che sento dentro di me. E, mentre ascolto il fruscio del vento tra le foglie, lo paragono a un altro suono che immagino: il silenzio infinito del tempo passato e del tempo futuro. In quel confronto tra il presente che sento e l’eternità che immagino, la mia mente si perde. Ma proprio questo smarrimento mi è dolce, perché mi fa sentire parte di qualcosa di immenso e senza confini."
manzoni_ai(inp, temperature=0.90, top_p=0.90, mnt=320)

USER PROMPT:
		C'è una siepe, sopra questo colle solitario, che mi impedisce di vedere una parte dell’orizzonte più lontano. Ma proprio
		perché non vedo, posso immaginare con la mia mente spazi infiniti oltre la siepe, silenzi profondi e pace assoluta, al
		punto che quasi mi viene da tremare: mi sembra di perdermi in quell’immensità che non riesco a vedere con gli occhi, ma
		che sento dentro di me. E, mentre ascolto il fruscio del vento tra le foglie, lo paragono a un altro suono che immagino:
		il silenzio infinito del tempo passato e del tempo futuro. In quel confronto tra il presente che sento e l’eternità che
		immagino, la mia mente si perde. Ma proprio questo smarrimento mi è dolce, perché mi fa sentire parte di qualcosa di
		immenso e senza confini.

MANZONI AI:
		L'esistenza d'una siepe, su questo colle solitario, che mi priva di una parte dell'orizzonte più lontano, non impedisce
		che, nel mio pensiero, io possa immaginare desolati spazi infiniti, silenzi infiniti, solenni